# STA 130 Lab 7: The Bootstrap
*Your Name Here*

The Central Limit Theorem was key in formulating the specific confidence interval for the estimate of the mean of a population we talked about in class.  For instance,  because the CLT says that the sample mean is normally distributed ($\bar{X} \sim N(\mu, \frac{\sigma}{\sqrt{n}})$), we can use the standard normal table to get critical values for the confidence interval.  These critical values scale the standard error of $\bar{X}$
(or any point estimator) to cover the desired level of confidence (e.g.  95%).  What if we didn’t have the CLT to tell us where to get these critical values from?  The goal of this lab is to investigate how to use bootstrapping to estimate confidence intervals.

## Bootstrapping

The method of bootstrapping (or bootstrap sampling) allows us to estimate the confidence
interval for a population parameter when we only know the estimator (a function) and have
a sample.  That is, we don’t know where to get our critical value from.  Finding bootstrap
confidence intervals generally goes like this:

1.  Get your sample of size $n$.
2.  Decide on a confidence level $(1-\alpha)$.
3.  Repeat $m$ times:
      (a)  Randomly draw a sample of size $n$ with replacement from your original sample (this is called a “bootstrap sample”).
      (b)  Calculate your point estimate of interest on this bootstrap sample.
4.  Look  at  the  distribution  of  your $m$ estimates  (the  “bootstrap  distribution”).   If  it is  roughly  symmetric,  you  may  proceed. (There  is  another  method  for  asymmetric distributions, but we won’t get into that here.)
5.  Order your $m$ bootstrap point estimates from lowest to highest.
6.  Your $100(\alpha/2)%$ lowest bootstrap estimate is your lower bound and your $100(1−\alpha/2)%$ lowest  estimate  is  your  upper  bound  on  your  confidence  interval.   Thus,  if  we  have 1,000 bootstrap samples and a 95% confidence interval,  our lower bound will be the 25th lowest estimate our upper bound will be the 975th lowest estimate (25th largest estimate).

## Why it works

Bootstrapping gives us a reliable estimate of the confidence interval for a population parameter because, with an unbiased estimator:
1.  Our sampling distribution is centered about our true population parameter value.
2.  Our bootstrap distribution is centered about our sample’s estimate of the true population value. 

Thus,  with  a  symmetric  bootstrap  distribution,  we  expect  our  bootstrap  distribution  to
have the same relationship to our one sample estimate as our sampling distribution (of the
estimator, if we could draw lots of real samples) would have to the true population value.

## Data
The RealEstate dataset (in excel and csv format) contains a collection of recent real estate listings in San Luis Obispo county.  The dataset contains the following fields:

- MLS: Multiple listing service number for the house (unique ID).
- Location:  city/town where the house is located.  In this dataset, all locations are in Santa Maria-Orcutt.
- Price:  the most recent listing price of the house (in dollars).
- Bedrooms:  number of bedrooms.
- Bathrooms:  number of bathrooms.
- Size:  size of the house in square feet.
- Price/SQ.ft:  price of the house per square foot.
- Status: type of sale.  Thee types are represented in the dataset: Short Sale, Foreclosure and Regular.

## Assignment
Focus on the “Price” variable:  return a maximum 2 page report to answer these questions. Write your own function to perform the bootstrap, do not use any Python libraries that perform bootstrapping for you. 

1. Draw a histogram and boxplot of the “Price” variable.  Comment on the distribution.
2. Mean of the housing prices:
    - Obtain  a  point  estimate  and  a  95%  CI  of  the mean from  the  sample,  using  the central limit theorem.
    - Find the standard error and 95% confidence interval of the estimate for mean of the  “Price”  variable  using  bootstrapping.   Use  different  number  of  iterations  for bootstrapping:  1000, 5000, 10000, and plot the obtained standard error against the number of iterations.  Compare the bootstrapping CIs with that obtained from the CLT.
3. Median of the housing prices:  Can we still use CLT to find the 95% estimate of the median of the prices?  Repeat problem 2(b) for median.